In [126]:
import numpy as np
import pandas as pd
import scipy

In [127]:
import subprocess
import threading
import sys
import os
import pandas as pd

In [128]:
mu = 1
alpha = 0.75
r = 0.02
sigma = 2
sigma_g = 2
omega_c = 0.3
pm = 1
gamma = 0.43
y_loss = 0.02
phi = 0.18
rho_a = 0.777
sigma_a = 0.029
mu_a = 0
zt = 0
tax_rate = 0.19
gn_min = 0
gn_max = 1 - 0.000001
h_min = 0.000001
h_max = 1
width = 1.5
ub = 1
lamb = 0.184
coup = 0.0
pi_number = 3.1415926535897932
kappa_val = 1000000
beta_hh = 1/(1+r)
chi_gn_hh = 0
alpha_c = 0.7
welfare_case = 0
only_repay = 1
tol = 0.000002
weight_q = 1
ns = 21
nk = 1
na = ns*nk
nb = 251
nn = 101
nmc = 10000
n0 = 1000
nb2 = 501
na2 = 21
n_devs = 3

nt = 9000
nt2 = 32
n_sam = 1000 
nm = 100
nf = 2    
max_iter = 500
n_iter_v = 15
n_cdf = 70
n_quad_v = 15
n_eps = n_quad_v
n_quad_q = 11
save_results = 0
choice_def_cost = 2
choice_counter = 0
choice_coup = 0
choice_bound = 1
choice_dist = 0
choice_nog = 0
choice_fe = 0
choice_now_bar = 0  #for choice_nowbar=1 set wbar=0d0
choice_correl = 0,
choice_tax_hat = 0                             #=0 no tax_hat, =1 frictionless tax level
choice_rule = 0                                #=0 use saved data, =1 use given coefficient for gN/yN
nn_aux = choice_fe*nn+(1-choice_fe)*1
export_data = 0              #=1 to export subsamples series
choice_util_hh = 1           #to compute value fcn for HH
choice_constant_gn = 1
choice_ols = 0
th_1 = -0.27982
th_2 = -1.21
min_gn = 0.045844
scale = 1.01
cdf_min = 3.167e-5
cdf_max=1+0-3.167e-5
huge_n=-1e6
resp_coef_l1=0.16
resp_coef_l2=0*0.4248
resp_coef_l3=0*0.539928
resp_coef_l4=0*(lamb+r)
resp_coef_h1=0.16
resp_coefh2=-0*0.297183
resp_coefh3=-0*0.257716
resp_coefh4=0*(lamb+r)
resp_coef1=0.200627
resp_coef2=-0.115658
resp_coef3=-0.026376*(lamb+r)
resp_coef4=0*(lamb+r)

In [129]:
b_min = -0.41
b_max = 0.35
hh_u_cost = 1
constant_gn_value = 0
coef_yt = 0
coef_bt = 0
x_val = [0.907, 0.02, 0.0, 0.3772, 2.42, 3.068]
dos = 2
eps_min = -4 * sigma_a
eps_max = 4 * sigma_a

In [130]:
points_df = pd.read_csv("/if/research-gms/William/grid_points/eighth/hyperplane_points_grid.csv")
planes_df = pd.read_csv("/if/research-gms/William/grid_points/eighth/plane_functions_parameters.csv")

In [131]:
quad_x_v, quad_w_v = np.polynomial.legendre.leggauss(15)
quad_x_q, quad_w_q = np.polynomial.legendre.leggauss(11)
quad_x_hermite, quad_w_hermite = np.polynomial.hermite.hermgauss(15)

In [132]:
un_cst_da = sigma_a / np.sqrt(1-rho_a**2)
a_min = mu_a / (1-rho_a) - n_devs * un_cst_da
a_max = mu_a / (1-rho_a) + n_devs * un_cst_da

a_grid = [a_min + (i)/(ns-1) * (a_max-a_min) for i in range(ns)]
gn_grid = [gn_min + (i)/(nn-1) * (gn_max-gn_min) for i in range(nn)]
h_grid = [h_min + (i)/(nn-1) * (h_max-h_min) for i in range(nn)]
cdf_grid = [cdf_min + (cdf_max-cdf_min) * (i)/(n_cdf-1) for i in range(n_cdf)]

eps_grid = [np.sqrt(dos) * sigma_a * quad_x_hermite[i] for i in range(n_eps)]

kw = omega_c / (1 - omega_c)
payoff = 1
if choice_coup == 1:
    payoff = lamb+ (1-lamb)*coup

default_grid = [0,1]
f_tol = 1e-3
xi = np.zeros((6, 6))
xi = np.array([[1.0 if j == i else 0.0 for j in range(1, 7)] for i in range(1, 7)])


## Government Function

In [133]:
# Compute bgrid
b_grid = [b_min + (b_max - b_min) * (i)/(nb-1) for i in range(nb)]

In [134]:
# Binary search algorithm - tested for below index 0 and above last index and it replicates LOCATE 
i0 = np.searchsorted(b_grid, 0, side="right")
b_grid[i0] = 0

In [135]:
beta = x_val[0]
chi = x_val[1]
i_cost = x_val[2]
psi_1 = x_val[3]
psi_2 = x_val[4]
w_bar = x_val[5]

In [136]:
v0_matrix = np.zeros((nb,na))
v0_matrix[1,1]

0.0

In [154]:
# INITIAL_GN
# Initialize terminal node with default as risk-free debt environment
q_matrix_nodef = np.zeros((na,nb))
q_matrix_nodef[:,:] = payoff / (lamb + r)
v0_matrix = np.zeros((nb,na))
vaut_matrix = np.zeros((nb,na))-1e-4
index_bfeas4=np.zeros(na)
f_data = np.zeros(nb)
break_matrix = np.zeros((na,nb))
coeff_matrix = np.zeros((na,4,nb))
# Compute spline for feasible debt gridpoints
for i_a in range(na):
    new_value=-1e5
    i_b=0
    index_opt = 0
        
    while new_value <= -1e3:
        index_opt = i_b
        if i_b == nb:
            break
        i_b = i_b+1
        new_value = v0_matrix[i_b,i_a]
    index_bfeas4[i_a]=index_opt   

for i_a in range(na):
    if index_bfeas4[i_a]+1 <= nb-1:
        ii_b = int(index_bfeas4[i_a]) + 1
        nbaux = nb - ii_b + 1
        f_data[1:nbaux] = v0_matrix[ii_b:nb, i_a]

        cs = scipy.interpolate.CubicSpline(
            x=b_grid[ii_b:nb], 
            y=f_data[1:int(nbaux)],
            bc_type = 'not-a-knot',
            )
        
        break_matrix[i_a,ii_b:nb], coeff_matrix[i_a,:,ii_b:nb-1] = cs.x, cs.c

for i_a in range(na):
    f_data = q_matrix_nodef[:,i_a]

In [155]:
f_data

array([4.90196078, 4.90196078, 4.90196078, 4.90196078, 4.90196078,
       4.90196078, 4.90196078, 4.90196078, 4.90196078, 4.90196078,
       4.90196078, 4.90196078, 4.90196078, 4.90196078, 4.90196078,
       4.90196078, 4.90196078, 4.90196078, 4.90196078, 4.90196078,
       4.90196078])

In [ ]:
for i_b in range(1,nb+1):
    for i_a in range(1,na+1):
        a_initial = agri